# Class 9 - 6.5.18

# Advanced and Performant Python

One of the best things about Python is how easy it is to get started with. The syntax is clear, it has all the basic features, things work as you expect them to, and life is generally pleasant. But Python also supports very advanced features, which make coding with Python an enjoyable experience even after you think you've learned everything there is to know of the language.

While technically you can write good Python code without using these features - it's sometimes a real shame not to use them.

## Generators

In a simple sense, perhaps simplistic, generators are iterators. Meaning, a generator is always an object you can iterate over. In Python you can iterate over most data structures, including dictionaries, lists, tuples and more - and so in this sense generators are similar. However, when we iterate over a list, for example, we're iterating over an existing list with existing items. Likewise for dictionaries - when we iterate over them, we're "handed" with the dictionary's keys and values.

This is the first major difference between a generator and the other iterators. A generator is a _recipe_ to create the next item in the chain. A generator is a piece of code telling the Python interpreter how to create the next item, but it doesn't hold this item in memory yet. A simple example might be a list containing values from 0 to 1000. A generator of this list will not have 1000 cells with their values - it would have instructions on the number of cells, and how to calculate the next value.

We've already met a generator (well, kind of) - the `range()` function. When we tell Python to give us a range of number between 0 and 1000 by writing `range(1000)` - we're not actually generating the 1000 "cells" of values, but only the recipe. Let's see it in "action":

In [1]:
range(1000)  # a "range" object

range(0, 1000)

In [2]:
items = range(1000)
items

range(0, 1000)

In [3]:
import sys
sys.getsizeof(items)  # 48 bytes - not nearly enough to hold 1000 items

48

A simple 1000-element list isn't that heavy for a computer (but what about Arduinos?), but when lists get longer, with bigger arrays and massive data structures inside them, it's very inefficient to hold this amount of unused data in memory. 

Let's define our own generator:

In [4]:
def my_range(n):
    """ Returns a list of items from 0 to n """
    num = 0
    while num < n:
        yield num
        num += 1

When we create a generator, the code is executed until the first `yield` statement. This reserved keyword is what makes a function into a generator.

When the code reaches the `yield` it holds, or "saves" its current state, until called by Python's `next()` function:

In [5]:
new_range = my_range(3)

print(next(new_range))

print(next(new_range))

print(next(new_range))

print(next(new_range))

0
1
2


StopIteration: 

Each time `next()` is used, the line with the `yield` is executed, and the function keeps going until it reaches  another `yield` statement. In the `my_range` function, while the index is smaller than `n` the code will reach a `yield`. When we don't satisfy this condition anymore, the code skips the loop and reaches the end of the function. This results in a special `StopIteration` exception, used only in these special cases. This means you can catch this exception and know that your generator went through all of its items.

But calling `next()` multiple times isn't practical. Luckily, `for` loops implement this exact interface automatically, allowing us to use them instead of the tedious, repetitive `next()` calls:

In [6]:
looprange = my_range(10)

for item in looprange:
    print(item)

0
1
2
3
4
5
6
7
8
9


The `for` loop is also smart enough to catch the `StopIteration` exception and terminate the loop, without raising any "visible" exceptions. A `for` loop is the common way to iterate over generators.

Generators don't allow much besides it. You can't print them exactly, or index into them:

In [7]:
range2 = my_range(10)
print(range2)

<generator object my_range at 0x000001AA598ADC50>


In [8]:
range2[3]

TypeError: 'generator' object is not subscriptable

Once used, generators are "depleted", you can't reuse them. This is a major difference between a generator and a list, for example - you're not limited by the number of times you can iterate over a list.

In [9]:
for item in looprange:
    print(item)
# Doesn't return anything, because we already depleted looprange

In [10]:
next(looprange)  # immediately raises StopIteration

StopIteration: 

It's important to stress that all functions can become generators if they contain the `yield` statement:

In [11]:
def println():
    print("Hello, ")
    yield True
    print("World")
    yield False

In [12]:
gen2 = println()
a = next(gen2)

print(a)

Hello, 
True


In [13]:
b = next(gen2)
print(b)

World
False


Another way to create generators is "genexps", or generator expressions, which are very similar to list comprehensions:

In [14]:
nums = (2 * n for n in range(10))
nums

<generator object <genexpr> at 0x000001AA598E55C8>

In [15]:
for num in nums:
    print(num)

0
2
4
6
8
10
12
14
16
18


The round brackets tell the interpreter that we're creating a generator here.

### Exercise
Write a generator function, or a piece of code including a generator, returning the `n` first Fibonacci numbers. The Fibonacci sequence starts with `1, 1` and the following item is always the addition of the last two items.

### Exercise solution below...

In [16]:
# Solution with a single function
def fib(n):
    """ Returns the first n Fibonacci numbers """
    a, b = 1, 1
    idx = 0
    while idx < n:
        yield a
        a, b = b, a + b
        idx += 1

ten = fib(10)
list(ten)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [17]:
# Solution as a script
def fibn():
    """ Runs over all Fibonacci numbers """
    a, b = 1, 1
    while True:
        yield a
        a, b = b, a + b
        
        
gen = fibn()
for i in range(10):
    print(next(gen))

1
1
2
3
5
8
13
21
34
55


The nice thing about this implementation is that it's infinite - it contains, and can generate all Fibonacci numbers. We could never do that with lists and regular functions.

Generators are used widely in the Python universe. `pathlib` uses them everywhere, for example. For our use-cases, which involve "data-science" stuff, it's usually less important. However, in one of my projects I have a large 5D array which I need to create. This array can easily weigh more than 1 GB RAM when I'm dealing with longer experiments. That's why I decided to write a generator function that creates and populates this array, `yield`ing 4D substacks of it over time. It reduces memory usage by at least two orders of magnitude, and allows my code to run on simpler machines.

## Decorators

Decorators are functions that receive functions in their arguments. When you wrap an existing function with another function - you created a decorator. This feature is extensively used in web frameworks and in other important Python use cases, which means it has a special syntax: `@decorator`. Let's look at an example:

Assume I have a large data-processing pipeline script, built out of many smaller functions, which unfortunately takes a long time to run. I wish to understand _why_ it's taking so long, so I decide to add a printed statement at the start and end of each function, so that I could see with my eyes where the code "hangs". This is how I implemented it:

In [18]:
def main_pipeline(fname):
    data = load_data(fname)
    processed = process_data(data)
    appended = append_data(processed)
    logged = log_data(appended)


def load_data(fname):
    print("Starting 'load_data'...")
    # ... Code ...
    print("Ending 'load_data'...")

def process_data(data):
    print("Starting 'process_data'...")
    # ... Code ...
    print("Ending 'process_data'...")
    
# And so on...

This is obviously very tedious. Even when I only have four functions, it's very repetitive and feels wrong. Moreover, it might have not solved my issue. My examination showed that all four functions take a considerable time to run, so I decide the profile the execution time of each function, to better understand which function is the most costly and optimize it first.

Here's how I redefined all functions to measure their execution time:

In [19]:
import time

def load_data(fname):
    print("Starting 'load_data'...")
    start_time = time.time()
    # ... Code ...
    print(f"It took the code {time.time() - start_time} milliseconds to run.")
    print("Ending 'load_data'...")

def process_data(data):
    print("Starting 'process_data'...")
    start_time = time.time()
    # ... Code ...
    print(f"It took the code {time.time() - start_time} milliseconds to run.")
    print("Ending 'process_data'...")
    
# And so on...

This works, but again - it's very repetitive. Also, if I decide that I want to see the execution time in seconds, and not milliseconds, I have to go through each function and re-implement it. Very tedious. 

The solutions is to _decorate_ the function with a `printer` and `timer` functions, that do this job exactly:

In [20]:
def printer(func):
    def inner_func(argument):
        print(f"Starting {func.__name__}...")
        result = func(argument)
        print(f"Ending {func.__name__}...")
        return result
    return inner_func      


def timer(func):
    def inner_func(argument):
        start_time = time.time()
        result = func(argument)
        print(f"It tooks the code {time.time() - start_time} milliseconds to run.")
        return result
    return inner_func

This looks complex at first, but it's really pretty simple. It uses the fact that functions in Python are objects, like any other element in the language. And because they're objects, they can be passed around as arguments:

In [21]:
def f(func):
    """ Runs the func functions and prints 'hi' at the end """
    func()
    print("hi")
    
def print_hello():
    print("hello")
    

f(print_hello)

hello
hi


Like all objects, functions have attributes. Namely, they have the `__name__` attribute which contains... their name.

In [22]:
print(f.__name__)
print(print_hello.__name__)

f
print_hello


Now we know we can pass functions as arguments to other functions. Let's try to examine the `printer` and `timer` functions again.

They're both a function that receives a different, "unknown" function, as its argument. So far - so good. Then it defines another function which "wraps" the original function with some actions, like printing or timing. This inner function runs the original function and returns the result. In essence, it created a "new implementation" of that original function that does the exact same thing, but with the wrapping functionality (printing, timing, etc.). This new function (`inner_func`) can replace any instance of the original function without any troubles, since in essence it just calls it. It's adds a couple of statements before and after that call, but the essential functionality remained unchanged.

Lastly, the outer function, which we call the decorator, returns the inner function as its return value. So this function receives a function as its argument and return a new, improved function as its output. To use it, we just "rename" the existing functions:

In [23]:
load_data_printer = printer(load_data)
load_data_timed = timer(load_data)

process_data_printer = printer(process_data)
process_data_timed = timer(process_data)

We can obviously use this `timer` function on any function we wish to time. When we wish to time functions in seconds, rather than milliseconds, we'll just change this one instance of `timer` and be done with it, and likewise for `printer`.

The only small caveat here is the fact that we currently require the function we're replacing to have a single `argument` as its argument. This implementation detail is small but very impactful - it means that our decorator will only decorate successfully functions that have a single argument. To remedy this we'll have to use `*args` and `**kwargs`:

### Detour - `*args`, `**kwargs`

You use `*args` and `*kwargs` when you're not sure how many arguments are used for a function. Actually, the syntax is only `*` and `**` - the words `args` and `kwargs` are used by convention. `args` is obviously arguments, or unnamed arguments given to a function. `kwargs` is keyword arguments, or arguments given as `key=value`. Let's see a simple example:

In [24]:
def f(required_argument, *args, **kwargs):
    print(required_argument)
    if args:
        print(args)
    
    if kwargs:
        for key, value in kwargs.items():
            print(key, value)

In [25]:
f()  # doesn't work - we have one required argument to the function

TypeError: f() missing 1 required positional argument: 'required_argument'

In [26]:
f('required')

required


In [27]:
f('required', 1, 2, 3)  # the second printed row is the args

required
(1, 2, 3)


In [28]:
f('required', 1, 2, 3, kw1='a', kw2='b')

required
(1, 2, 3)
kw1 a
kw2 b


We see that `args` is a tuple containing all unnamed parameters that were given to the function, in the order they were given.

`kwargs` is a dictionary, its keys being the keywords, and values - the given values. Here's another short example:

In [29]:
def f(a=1, b=2):
    print(a, b)

inputs = {'a': 10, 'b': 20}
f(**inputs)

10 20


What we see here is that the function's signature doesn't have to contain `*args` or `**kwargs`. The `**` operator "opens up" the input dictionary, allowing the `f()` function to use the parameters without any issues. This is how we're going to use it for our decorators - we'll redefine them as follows:

In [30]:
def printer(func):
    def inner_func(*args, **kwargs):
        print(f"Starting {func.__name__}...")
        result = func(*args, **kwargs)
        print(f"Ending {func.__name__}...")
        return result
    return inner_func      


def timer(func):
    def inner_func(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        print(f"It tooks the code {time.time() - start_time} milliseconds to run.")
        return result
    return inner_func

We can now be sure that our functions will always run regardless the number of inputs given to them. This makes us happy - but not completely happy. We still have to redefine all functions as we've seen before:

In [31]:
load_data_printer = printer(load_data)
load_data_timed = timer(load_data)

process_data_printer = printer(process_data)
process_data_timed = timer(process_data)

It still requires us to rename all instances of these functions in all places of the code, and when we're done with the printing and timing - we have to rename them back.

Why not "rename" the function back to its original name?

In [32]:
load_data = printer(load_data)
process_data = timer(process_data)

This idiom is common enough to have a built-in language syntax:

In [33]:
@timer
def load_data(fname):
    # ... Code ...
    pass

We can use multiple decorators for functions as well:

In [34]:
@printer
@timer
def process_data(data):
    # ... Code ...
    pass

When we wish to stop printing and timing our function, we simply delete this decorator in the relevant places.

Decorators allow more complex calls, like calling them with arguments, but we'll leave that topic for another day.

## `collections`, `itertools`

The Python standard library comes with many second-order tools that can make your life much easier. Many of the more useful ones are located in these two libraries - `collections` and `itertools`. Below I'll provide a brief tour of some of the more interesting features of these libraries.

### namedtuple

When you want a tiny object with named fields, but without the hassle of creating a fully-fledged class, you actually wish to generate a namedtuple:

In [35]:
from collections import namedtuple

Point = namedtuple('Point', ['x', 'y'])
p1 = Point(0, 0)
p2 = Point(x=0, y=1)
p3 = Point(1, y=2)

print(p2)
print(p3.y)
print(p1[0])

Point(x=0, y=1)
2
0


You can access the data inside a `namedtuple` using either the positional index (`[0]`) or the name of that field (`x`). If all you wish to do is to a keep a small record of something, `namedtuple` is your best option.

### defaultdict

A `defaultdict` is a dictionary that resorts to execute a predefined function if it doesn't find the key. For example:

In [36]:
d = dict(one=1, two=2)
print(d['one'])
print(d['three'])

1


KeyError: 'three'

Rather than a `KeyError`, a `defaultdict` would run a predefined function instead of raising this exception:

In [37]:
from collections import defaultdict

d2 = defaultdict(list, one=1, two=2)
print(d2['one'])

1


However, when we call it with an unknown key:

In [38]:
d2['three']
d2

defaultdict(list, {'one': 1, 'three': [], 'two': 2})

It used the `list` "factory" to create a new list in that key. This is useful when sorting some key-value pairs.

In [39]:
s = [('yellow', 1), ('blue', 2), ('yellow', 3), ('blue', 4), ('red', 1)]
d2 = defaultdict(list)
for k, v in s:
    d2[k].append(v)

d2

defaultdict(list, {'blue': [2, 4], 'red': [1], 'yellow': [1, 3]})

### Chained iterables

If we wish to iterate over several iterables together, we can use the following method from the `itertools` module:

In [40]:
import itertools

chained = itertools.chain('abcd', 'efg')
for letter in chained:
    print(letter)

print('-----')
# Naive iteration over [[1, 2, 3, 4], [5, 6, 7, 8]] would result in two items - 
# two lists with four elements each. We wish to iterate over the number themselves.
chained2 = itertools.chain.from_iterable([[1, 2, 3, 4], [5, 6, 7, 8]])
for letter in chained2:
    print(letter)

a
b
c
d
e
f
g
-----
1
2
3
4
5
6
7
8


Note that `itertools` always creates generators from the items it receives as input.

### Permutations

In [41]:
list(itertools.permutations('ABCD', 2))

[('A', 'B'),
 ('A', 'C'),
 ('A', 'D'),
 ('B', 'A'),
 ('B', 'C'),
 ('B', 'D'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'D'),
 ('D', 'A'),
 ('D', 'B'),
 ('D', 'C')]

### Combinations

In [42]:
list(itertools.combinations('ABCD', 2))

[('A', 'B'), ('A', 'C'), ('A', 'D'), ('B', 'C'), ('B', 'D'), ('C', 'D')]

## Multiprocessing

There are several ways to utilize parallel processing in Python. The easiest of all is multi-processing, i.e. the use of several CPU cores to run jobs in parallel. This is best used when each process is independent from the others, not having to share data between them. 

A typical use case is when we have a list or an `np.array` holding data, and we wish to perform the same computation on each element of that list. If this computation is truly independent, the `multiprocessing` module has some very easy-to-use solutions.

```python
import multiprocessing

def add_tuple(tup):
    return tup[0] + tup[1]

tups = [(0, 1), (2, 3), (4, 5), (6, 7)]
pool = multiprocessing.Pool()  # can also enter the number of processes you wish to use
result = pool.map(add_tuple, tups)
result  # [1, 5, 9, 13]
```

The code above doesn't work in IPython and Jupyter due to some weird conflicts. Luckily, `ipyparallel` is an even better library which does the exact same thing and works everywhere.

The Python script `multiprocess.py` located in this `Classes` folder contains a working copy of this script.

Threading is Python's weak point because of the GIL, and we'll not discuss it in this class. Another form of parallel processing is asynchronous programming, which we'll also not cover, but is actually one of Python's strongest points.

## Numba

`numba` is a special library designed to speed-up Python's computation. In many cases it's comparable to `numpy` in terms of use cases, but it might be simpler for people without previous experience with arrays. We'll jump right into an example and then discuss some of the magic:

In [43]:
from numba import jit
import numpy as np


@jit
def sum2d(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

In [44]:
arr = np.ones((10000, 10000))

%timeit sum2d(arr)
%timeit arr.sum()

196 ms ± 65.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
163 ms ± 8.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


The results up there are, for lack of a better word, amazing. Numpy has been optimized for ages, works in bare C, and is still slightly topped by `numba`, which seemingly just decorates a simple, perhaps _simplistic_, Python loop, making it amazingly fast.

This magic happens with LLVM, an open-source project that aims at building a very fast, cross-language compiler. `numba` translates the code to LLVM-suitable code and lets LLVM optimize this code for it. The output is machine code which is fed into the processor directly, and somehow it's faster than all other solutions.

Numba has more tricks in its sleeve. You can define the input types to squeeze it a bit more:

In [45]:
from numba import jit, float64
import numpy as np


@jit(float64(float64[:, :]))
def sum2d_inps(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

In [46]:
%timeit sum2d_inps(arr)

143 ms ± 7.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


We can also use parallel looping:

In [47]:
from numba import jit, prange
import numpy as np


@jit([float64(float64[:, :])], parallel=True)
def sum2d_p(arr):
    M, N = arr.shape
    result = np.float64(0.0)
    for i in prange(M):
        for j in prange(N):
            result += arr[i,j]
    return result

In [48]:
%timeit sum2d_p(arr)  # pretty cool

62.2 ms ± 6.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


When every bit of performance matters - `numba` might be the way to go. For very complicated functions that use fancy linear algebra algorithms, it might be the case that `numba` doesn't support these methods yet. In these occasions resort to basic `numpy` functions and wait till the `numba` developers implement that method - or do so yourself! `numba` is completely open-sourced.

## Cython

When you wish to write performant code that utilizes significant parts of the standard library - niether `numpy` nor `numba` will help you. They require that you work with arrays, which are not as easy to work with as lists, for example. Dictionaries are also very helpful, but using them only with the standard Python interpreter will hinder you performance considerably.

These are the cases where Cython shines. It allows you to write code with Python-like syntax and compile it ahead-of-time to a `myfile.c` source file, written in `C` automatically. When your code calls a function that was written in Cython, it will actually turn to the optimized `C` function and use that function instead.

As stated, Cython requires you to compile your code before running the parent Python script. To do that, you have to create a `setup.py` file that tells the Cython compiler where to find the files in question.

A Cython file ends with `X.pyx`, so `setup.py` should point there. Here's a basic example of `setup.py`:

```python
from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize('my_file.pyx'),
    # other setup.py options come here
)
```

Then you navigate with your command line to the folder containing `setup.py` and write `python setup.py build_ext --inplace`, which tells Cython to "build", i.e. compile, the code in the `.pyx` file and add it `inplace`, i.e. to this directory.

An example can be found in the `cython_demo` folder. Let's see it here in action:

In [49]:
from cython_demo import plain_python, primes_cython

In [50]:
plain_python.primes_python(20)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]

In [51]:
primes_cython.primes(20)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]

In [57]:
%timeit plain_python.primes_python(1000)
%timeit primes_cython.primes(1000)

47.8 ms ± 5.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
2.36 ms ± 341 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [53]:
rands = np.random.random((1000000))

In [54]:
%timeit rands[rands < 0.5]

13.8 ms ± 836 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [55]:
@jit([float64[::1](float64[::1])], nopython=True, parallel=True)
def filter_larger(rands):
    arr = np.zeros_like(rands)
    thresh = 0.5
    last_idx = 0
    for idx in prange(len(rands)):
        if rands[idx] < 0.5:
            arr[last_idx] = rands[idx]
            last_idx += 1
            
    return arr[:last_idx]

# The last_idx variable is probably hindering performance of the parallel loop

In [62]:
%timeit filter_larger(rands)

13.2 ms ± 1.34 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [60]:
from cython_filter_demo import filter_array

In [61]:
%timeit filter_array.filter_larger_cython(rands)

187 ms ± 54.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [64]:
%prun filter_array.filter_larger_cython(rands)